In this notebook I will try to read pdf files and extract text from them.

I have a bunch of pdf files and I would like to extract some text data from them , here is how I will handle that

In [ ]:
# Run the following scripts before processing
! rm data/raw/text_from_pdf/*.txt

In [1]:
from pathlib import Path

In [2]:
import fitz as pdf_extractor

In [3]:
PDF_FILES_PATH = Path.cwd().parent.joinpath('data' , 'raw', 'pdf')

In [4]:
all_pdf_files =  [x for x in PDF_FILES_PATH.glob('*.pdf') if x.is_file()]

In [5]:
all_pdf_files

[PosixPath('/Users/es.py/Projects/Personal/multilingual-drc-news-chatbot/data/raw/pdf/constitution-en-lingala.pdf'),
 PosixPath('/Users/es.py/Projects/Personal/multilingual-drc-news-chatbot/data/raw/pdf/Manuel_in_lingala.pdf'),
 PosixPath('/Users/es.py/Projects/Personal/multilingual-drc-news-chatbot/data/raw/pdf/LOLENGE EKOLO CONGO EZALI KOMILENGELE PE EKANI KOSALELA REDD+ (RPP) Bomoko_ Mosala _ Tombwa - PDF Free Download.pdf'),
 PosixPath('/Users/es.py/Projects/Personal/multilingual-drc-news-chatbot/data/raw/pdf/CV_a-first-look-pamphlet-Lingala.pdf'),
 PosixPath('/Users/es.py/Projects/Personal/multilingual-drc-news-chatbot/data/raw/pdf/lingala.pdf'),
 PosixPath('/Users/es.py/Projects/Personal/multilingual-drc-news-chatbot/data/raw/pdf/fcpa-lingala.pdf'),
 PosixPath('/Users/es.py/Projects/Personal/multilingual-drc-news-chatbot/data/raw/pdf/Album village assaini Lingala.pdf'),
 PosixPath('/Users/es.py/Projects/Personal/multilingual-drc-news-chatbot/data/raw/pdf/lingala-faqs-4-8.pdf'),
 

In [6]:
manuel_in_lingala_path  = all_pdf_files[0]

In [7]:
def parse_text_from_html(path):
    """
    parse html text from the document passed in parameter
    """
    with pdf_extractor.open(path) as doc:
        raw_text = ""
        for page in doc:
            content = page.get_text("text")
            raw_text = f"{raw_text} \n{content}"
        return raw_text

In [8]:
def write_output(path, content, name):
    with open(path.joinpath(name), "w") as output_file:
        output_file.write(f"{content}\n")

In [9]:
OUTPUT_PATH = PDF_FILES_PATH.parent.joinpath("text_from_pdf")

In [10]:
OUTPUT_PATH.mkdir(exist_ok=True)

In [11]:
OUTPUT_PATH.name

'text_from_pdf'

In [12]:
for file in all_pdf_files:
    raw_text = parse_text_from_html(file)
    file_name = file.name.__str__().replace('pdf', 'txt')
    write_output(OUTPUT_PATH, raw_text, file_name)

### Cleaning the file

In this step we will try to clean the annodate corpus to make it a cleanned corpus , we will again use polygot to sptlit text into sentence and use some regular expressions to make our text data cleanned

In [13]:
TEXT_PATH = OUTPUT_PATH

All the files are differents in the formating but here are some steps we can perform to clean them all in one
-  remove number in a staring lines
- use polyglot to split the data into sentences
- remove (a) (B) and other similar sentence in the begining of sentences
- we can also remove urls 

In [14]:
all_files = TEXT_PATH.glob('*.txt')
with open(OUTPUT_PATH.joinpath('all.txt'), "w") as outfile:
    for file in all_files:
        with open(file, "r") as infile:
            outfile.write(f"{infile.read()}\n")

In [15]:
list(TEXT_PATH.glob('*.txt'))

[PosixPath('/Users/es.py/Projects/Personal/multilingual-drc-news-chatbot/data/raw/text_from_pdf/all.txt'),
 PosixPath('/Users/es.py/Projects/Personal/multilingual-drc-news-chatbot/data/raw/text_from_pdf/LOLENGE EKOLO CONGO EZALI KOMILENGELE PE EKANI KOSALELA REDD+ (RPP) Bomoko_ Mosala _ Tombwa - PDF Free Download.txt'),
 PosixPath('/Users/es.py/Projects/Personal/multilingual-drc-news-chatbot/data/raw/text_from_pdf/Manuel_in_lingala.txt'),
 PosixPath('/Users/es.py/Projects/Personal/multilingual-drc-news-chatbot/data/raw/text_from_pdf/constitution-en-lingala.txt'),
 PosixPath('/Users/es.py/Projects/Personal/multilingual-drc-news-chatbot/data/raw/text_from_pdf/LIN58-0720M By Faith Moses VGR.txt'),
 PosixPath('/Users/es.py/Projects/Personal/multilingual-drc-news-chatbot/data/raw/text_from_pdf/lingala-faqs-4-8.txt'),
 PosixPath('/Users/es.py/Projects/Personal/multilingual-drc-news-chatbot/data/raw/text_from_pdf/CV_a-first-look-pamphlet-Lingala.txt'),
 PosixPath('/Users/es.py/Projects/Person

In [16]:
def remove_urls(sample):
    """Remove URLs from a sample string"""
    sample = re.sub(r"\({0,}http\S+\){0,}", "", sample)
    sample = re.sub(r"\({0,}www\S+\){0,}", "", sample)
    return sample

In [17]:
import re

In [66]:
raw_text = ''
with open(OUTPUT_PATH.joinpath('all.txt'), 'r') as infile:
    for line in infile.readlines():
        line = remove_urls(line)
        line = re.sub(r'^(\d\.){1,}', '', line) # stuftf like 1.1 1.3
        line = re.sub(r'^\d{1,}', '', line) # line starting with digit 1
        line = re.sub(r'^\({1,}\w+\)','', line)#(i), (A)
        line = re.sub(r'^\({1,}\w+\)','', line) # samething twicet
        line = re.sub(r'^\W-\s', '', line)# remove any non white space char ignoring line
        line = re.sub(r'^•', '', line)
        line = re.sub(r'^-', '', line)
        line = re.sub(r'QUESTION \d+: ', '', line)
        line = re.sub(r'REPONSE \d+: ', '', line)
        line = re.sub(r'Atikele \d+ : ', '', line)
        raw_text += f"{line}\n"
        #print(line)
        #print(line)

In [89]:
sample_text = 'REPONSE 1: Mama moko azali kobuaka matiti na likbulu.'
re.sub(r'REPONSE \d+: ', '', sample_text)

'Mama moko azali kobuaka matiti na likbulu.'

In [67]:
text_string =  '- Kosolola na libota na lisalisi ya moninga moko ya libota ya babuki mibeko '

In [68]:
re.sub(r'^\W_-\s', '', text_string)

'- Kosolola na libota na lisalisi ya moninga moko ya libota ya babuki mibeko '

In [69]:
from polyglot.detect.base import UnknownLanguage
from polyglot.detect import Detector
from polyglot.text import Text

In [70]:
PROCESSED_FILE_PATH = OUTPUT_PATH.parent.parent.joinpath('processed', 'from_pdf_cleanned.txt')

In [71]:
CLEAN_FILE_PATH = OUTPUT_PATH.joinpath('all_cleanned.txt')

In [72]:
def polygot_cleaning(text):
    try:
        detector = Detector(text)
        language = detector.language.code
        print(detector.language)
        if language == 'ln':
            sentences = Text(text).sentences
            for sentence in sentences:
                if len(sentence.split(' ')) >= 3:
                    with open(CLEAN_FILE_PATH, "a") as output_file:
                        output_file.write(f"{sentence.string}\n")
    except UnknownLanguage:
        print(text)

In [73]:
def read_file(path):
    with open(path , 'r') as file:
        content = file.read()
        return content

In [74]:
polygot_cleaning(raw_text)

name: Lingala     code: ln       confidence:  98.0 read bytes:  1060


By end of the day the text is not well cleanned , we cand do better next time.

In [75]:
cleanned_text = read_file(CLEAN_FILE_PATH)

In [76]:
cleanned_text

"Makambo makomami na buku oyo ya sika ya R-PP Congo masembolami na ndenge ya polele polele\nEpusi bozongeli mbala na mbala matangi masusu to milongo na yango po na biteni bisusu oyo\nMataki ya koyangela REDD+ ;\nTina ya makambo eutaka na bokati banzete pambapamba pe kobebisa bazamba ;\nKosangisa makambo ya motuya oyo euti na makanisi ya IIASA pe eulani na oyo ya\nCONGOBIOM bakisa masengi ma linaka ya REDD+ ya bikolo bia Africa ya katikati oyo\nbazwaka lisalisi na misolo mia FEM (GEF), oyo Banque Mondiale na COMIFAC bapesaka.\nMRV ya CONGO po na kosalela makinisi oyo ebongisamaki po na MRV ya ba mboka ba\nkatikati to MRV elongo na FAO na misolo oyo CBFF apesaka;\n« budget »mobimba na ba «budget» misusu oyo etali biteni binso.\nSoki tososoli malamu, makambo mingi esi esalami.\ntozali kosenga na molongo\nmobimba bandima lolenge na biso ya kosalela REDD+ po ete basalisa biso po na makanisi na\nmayele oyo tosikongolo na kombo ya ekolo Congo , epusi na makambo etali boyangeli bazamba na\nnde

We can try to use the english tokeniser to try to split out our text intot sentences and again for long sentence use polygot again to split them into smaller sentences 

In [77]:
from nltk import tokenize

In [78]:
sentences = tokenize.sent_tokenize(cleanned_text)

In [79]:
len(sentences)

7851

In [80]:

filtered_sentences = set(filter(lambda sentence: len(sentence.split(" ")) >= 3, sentences))

In [81]:
len(filtered_sentences)

2845

In [82]:
import pandas as pd

In [83]:
sentences_df = pd.DataFrame(filtered_sentences, columns=['sentences'])

In [84]:
PROCESSED_FILE_PATH

PosixPath('/Users/es.py/Projects/Personal/multilingual-drc-news-chatbot/data/processed/from_pdf_cleanned.txt')

In [85]:
sentences_df.sentences = sentences_df.sentences.str.replace('\n', ' ').replace('\t', ' ')

In [86]:
sentences_df = pd.DataFrame(filtered_sentences, columns=['sentences'])

In [87]:
sentences_df.sentences = sentences_df.sentences.str.replace('\n', ' ').replace('\t', ' ').replace('"', '')

In [88]:
sentences_df.sentences.to_csv(PROCESSED_FILE_PATH, header=False, index=False)

The cleanning is done but a lot can be done manually in the editor tot make sure we have a cleanned file